# Module 1: Basics - Creating and Using Chains

Chains are sequences of calls to LLMs or other utilities.

This module covers:
1. Simple chains (LLM + Prompt)
2. Sequential chains
3. Custom chains
4. Batch processing

## Setup

In [ ]:
from langchain_ollama import ChatOllama
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from dotenv import load_dotenv
import os

load_dotenv()

# Initialize local Ollama model (no API key needed!)
llm = ChatOllama(
    model="moondream:latest",
    temperature=0.7,
    base_url="http://localhost:11434"
)

print("Setup complete! Using local Ollama model: moondream")

## Example 1: Simple Chain (LLM + Prompt)

The simplest chain combines a prompt template with an LLM using the `|` (pipe) operator.

This is the modern **LCEL (LangChain Expression Language)** way to create chains.

In [ ]:
# Create a prompt template
prompt = ChatPromptTemplate.from_template(
    "Explain {topic} in simple terms."
)

# Create a chain using the pipe operator (LCEL)
chain = prompt | llm | StrOutputParser()

# Run the chain
response = chain.invoke({"topic": "blockchain"})

print(f"Topic: blockchain")
print(f"\nResponse: {response}")

### Try different topics!

In [ ]:
# Try with a different topic
topics = ["quantum computing", "neural networks", "cloud computing"]

for topic in topics:
    response = chain.invoke({"topic": topic})
    print(f"\n📌 {topic.upper()}")
    print(f"{response[:200]}...\n")
    print("-" * 50)

## Example 2: Sequential Chain

Sequential chains pass the output of one step to the next. Let's create a chain that:
1. Generates a story
2. Summarizes the story

In [ ]:
# First chain: Generate a story
story_prompt = ChatPromptTemplate.from_template(
    "Write a very short story (2-3 sentences) about {topic}."
)
story_chain = story_prompt | llm | StrOutputParser()

# Second chain: Summarize the story
summary_prompt = ChatPromptTemplate.from_template(
    "Summarize this story in one sentence: {story}"
)
summary_chain = summary_prompt | llm | StrOutputParser()

# Combine them: story output becomes summary input
full_chain = (
    {"story": story_chain}
    | summary_chain
)

# Run the combined chain
topic = "a robot learning to paint"
story = story_chain.invoke({"topic": topic})
summary = full_chain.invoke({"topic": topic})

print(f"Topic: {topic}")
print(f"\n📖 Story:\n{story}")
print(f"\n📝 Summary:\n{summary}")

## Example 3: Chain with Multiple Variables

Chains can handle multiple input variables.

In [ ]:
# Create a chain that uses multiple variables
multi_var_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that formats information clearly."),
    ("human", """
    Topic: {topic}
    Format: {format}
    
    Provide information about the topic in the requested format.
    """)
])

format_chain = multi_var_prompt | llm | StrOutputParser()

result = format_chain.invoke({
    "topic": "Python programming",
    "format": "3 bullet points"
})

print("Topic: Python programming")
print("Format: 3 bullet points")
print(f"\nResponse:\n{result}")

## Example 4: Chain with RunnablePassthrough

`RunnablePassthrough` allows you to pass data through unchanged while also running other operations.

In [ ]:
# Chain that keeps the original input and adds processed data
prompt = ChatPromptTemplate.from_template(
    "What is {concept}? Explain briefly."
)

chain_with_passthrough = (
    {"concept": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Now you can pass the concept directly as a string
result = chain_with_passthrough.invoke("machine learning")
print(result)

## Example 5: Batch Processing

Process multiple inputs at once using the `batch()` method.

In [ ]:
# Create a simple explanation chain
explain_prompt = ChatPromptTemplate.from_template(
    "Explain {concept} in one sentence."
)

explain_chain = explain_prompt | llm | StrOutputParser()

# Process multiple concepts at once
concepts = ["machine learning", "neural networks", "deep learning"]
inputs = [{"concept": c} for c in concepts]

# Batch process all inputs
results = explain_chain.batch(inputs)

print("Batch processing results:\n")
for concept, result in zip(concepts, results):
    print(f"📌 {concept}:")
    print(f"   {result}\n")

## Example 6: Streaming Output

For long responses, you can stream the output token by token.

In [ ]:
# Create a chain for longer content
story_prompt = ChatPromptTemplate.from_template(
    "Write a short paragraph about {topic}."
)

stream_chain = story_prompt | llm | StrOutputParser()

# Stream the output
print("Streaming response:\n")
for chunk in stream_chain.stream({"topic": "the future of AI"}):
    print(chunk, end="", flush=True)
print("\n\n✅ Stream complete!")

## Key Takeaways

- **Chains** combine prompts and LLMs into reusable pipelines
- Use the **pipe operator** `|` for LCEL (LangChain Expression Language)
- **Sequential chains** pass output from one step to the next
- **batch()** processes multiple inputs efficiently
- **stream()** shows output as it's generated
- **RunnablePassthrough** passes data through unchanged

## Exercise: Build Your Own Chain

Create a chain that:
1. Takes a topic
2. Generates 3 questions about it
3. Answers one of those questions

In [ ]:
# Your turn! Build a custom chain

# Step 1: Generate questions
question_prompt = ChatPromptTemplate.from_template(
    "Generate 3 interesting questions about {topic}. List them numbered."
)
question_chain = question_prompt | llm | StrOutputParser()

# Step 2: Answer the first question
answer_prompt = ChatPromptTemplate.from_template(
    "Here are some questions:\n{questions}\n\nAnswer the first question briefly."
)
answer_chain = answer_prompt | llm | StrOutputParser()

# Combine them
full_chain = (
    {"questions": question_chain}
    | answer_chain
)

# Test it
topic = "space exploration"  # Change this!
questions = question_chain.invoke({"topic": topic})
answer = full_chain.invoke({"topic": topic})

print(f"Topic: {topic}\n")
print(f"Questions:\n{questions}\n")
print(f"Answer to first question:\n{answer}")